In [1]:
import datetime, os, pandas as pd, re, requests, time

In [2]:
today = datetime.date.today().strftime("%Y%m%d")
print (f'Сегодняшняя дата: {today}')
print (f'Сегодняшний год: {today[:4]}')
print (f'Сегодняшний месяц: {today[4:6]}')
print (f'Сегодняшний день: {today[6:]}')
path = r"/Users/yanny/Desktop/Python"

Сегодняшняя дата: 20231015
Сегодняшний год: 2023
Сегодняшний месяц: 10
Сегодняшний день: 15


In [3]:
def clndr_mdl(today):
    calendar = pd.DataFrame()
    month_n31 = ['02', '04', '06', '09', '11']
    for month in range(1, 13):
        month = '0' + str(month) if len(str(month)) == 1 else str(month)
        if month == '02':
            for day in range(1, 29):
                day = '0' + str(day) if len(str(day)) == 1 else str(day)
                calendar.loc[day, month] = day
            if int(today[:4]) % 4 == 0:
                calendar.loc['29', month] = '29'
        elif month in month_n31:
            for day in range(1, 31):
                day = '0' + str(day) if len(str(day)) == 1 else str(day)
                calendar.loc[day, month] = day
        else:
            for day in range(1, 32):
                day = '0' + str(day) if len(str(day)) == 1 else str(day)
                calendar.loc[day, month] = day
    return calendar

In [4]:
df = pd.DataFrame()
year_shift = 0

while year_shift >= 0:
    calendar = clndr_mdl(str(int(today[:4]) - year_shift) + today[4:])
    
    if year_shift == 0:
        months_for_search = calendar.columns[:int(today[4:6])]
    elif year_shift > 0:
        months_for_search = calendar.columns 
    else:
        print ('Something is wrong')
    
    for j in months_for_search:
        month = months_for_search[-int(j)]
        for i in calendar[month].dropna().index:
            day = calendar[month].dropna().index[-int(i)]
            AGR_file_name = path + f'/FL_конкурсы_{str(int(today[:4]) - year_shift)}{month}{day}_Агрегированный.xlsx'
            file_name = path + f'/FL_конкурсы_{today[:4]}{month}{day}.xlsx'
            if os.path.exists(file_name):
                df = pd.concat([df, pd.read_excel(file_name, index_col=0)])
            if os.path.exists(AGR_file_name):
                break
        
        if os.path.exists(AGR_file_name):
            print('!!!', f'The file {AGR_file_name} exists', sep='\n')
            break

    if os.path.exists(AGR_file_name) == False:
        print('Alas', f'The aggregated file does NOT exist in the {str(int(today[:4]) - year_shift)} year', sep='\n')
        print('???', 'Do you want to try the previous year?', sep= '\n')
        dcsn = input(' Press Enter. If you do not, input any symbol and then press Enter: ')
        if len(dcsn) == 0:
            year_shift += 1
        else:
            year_shift = -1
    else:
        df = pd.concat([df, pd.read_excel(AGR_file_name, index_col=0)])
        break

df = df.groupby(level=0).first()

df.to_excel(f'{path}/FL_Конкурсы_{today}_Агрегированный.xlsx', sheet_name='FL_конкурсы')

Alas
The aggregated file does NOT exist in the 2023 year
???
Do you want to try the previous year?
 Press Enter. If you do not, input any symbol and then press Enter: 
Alas
The aggregated file does NOT exist in the 2022 year
???
Do you want to try the previous year?
 Press Enter. If you do not, input any symbol and then press Enter: b
